In [ ]:
from collections import defaultdict
import numpy as np
import os
import json
import itertools
import networkx as nx
import pandas as pd
import random
import collections
from time import time
import xgi
import math
from tqdm import tqdm
import pickle as pk

def discrete_SIS(H,l,mu,dt_inf,run_id,theta,initial_number=1,tmin=0,tmax=float("Inf"),dt=1.0):
    #Simulates the discrete SIS model with threshold higher-order contagion on hypergraphs  
    #H - xgi.Hypergraph - The hypergraph on which we simulate the SIS contagion process
    #l - float - Infection probability of the model
    #mu - float - Healing rate
    #dt_inf - dictionary - keys are nodes' IDs, values are array where the time passed infected will be stored for each run
    #run_id - integer - ID of the actual realization of the simulation
    #theta - float - Threshold for hyper-infection
    #initial_number - int - Number of initially infected nodes
    #tmin - float - Time at which the simulation starts.
    #tmax - float - Time at which the simulation terminates (if it doean't reach the absorbin state)
    #dt - float - Time-step of the simulation.

    initial_infecteds = random.sample(list(H.nodes), initial_number)

    status = defaultdict(lambda: "S")
   
    for node in initial_infecteds:
        status[node] = "I"

    I = [len(initial_infecteds)]
    S = [H.num_nodes - I[0]]
    
    times = [tmin]
    t = tmin
    t_last_inf={}

    new_status = status.copy()
    
    det_act=0
    t_wait=100
    k_tstep=10
    
    T=tmax
    
    inf_new=[]
    inf_size=[]
    
    while t <= T:
        t += dt
        
        if det_act==0 and t>t_wait and np.max(np.abs(I[len(I)-k_tstep:len(I)]-np.mean(I[len(I)-k_tstep:len(I)])))<0.05*N: # the infection time is stored only when the system reach the steady state
            T=tmax+t
            det_act=1
            for node in H.nodes:
                if status[node] == "I":
                    t_last_inf[node]=t                    
        
        if I[-1]==0:
            initial_infecteds = random.sample(list(H.nodes), 1)
            status[initial_infecteds[0]] = "I"
            new_status = status.copy()
            I.append(len(initial_infecteds))
            S.append(H.num_nodes - 1)   
            if det_act==1:
                t_last_inf[initial_infecteds[0]]=t-dt
        else:      
            S.append(S[-1])
            I.append(I[-1])
        
        i_edge={}
        for edge_id in H.edges:
            edge = H.edges.members(edge_id)
            i_edge[edge_id]=0
            idx_s=[]
            for i in edge: #in each hyperedge estimate the number of I and the identitiy of nodes who are still S
                if status[i]=="I":
                    i_edge[edge_id]=i_edge[edge_id]+1
                elif new_status[i]=="S":
                    idx_s.append(i)
            
            if len(idx_s)>0 and (i_edge[edge_id]>=math.ceil(theta*len(edge))) and (random.random() <= l*dt): #conditions for group infection in the considered hyperedge (hihger-order threshold)
                for i in idx_s:
                    new_status[i]="I"
                    if det_act==1:
                        t_last_inf[i]=t
                        inf_size.append(len(edge))
                        inf_new.append(len(idx_s))
                I[-1] += len(idx_s)
                S[-1] += -len(idx_s)
                    
        for node in H.nodes:
            if status[node] == "I" and random.random() <= mu * dt: #recovery
                new_status[node] = "S"
                I[-1] += -1
                S[-1] += 1
                if det_act==1:
                    dt_inf[node][run_id]=dt_inf[node][run_id]+(t-t_last_inf[node])
        
        times.append(t)
        status = new_status.copy()        
    
    return np.array(times), np.array(S), np.array(I), inf_new, inf_size

In [ ]:
dataset='congress-bills_simplices';name_file='{0}'.format(dataset);
#dataset='senate-bills_simplices';name_file='{0}'.format(dataset);
#dataset='senate-committees_simplices';name_file='{0}'.format(dataset);
#dataset='house-committees_simplices';name_file='{0}'.format(dataset);
#dataset='email-Enron_simplices'; name_file='{0}'.format(dataset); 
#dataset='email-Eu_simplices'; name_file='{0}'.format(dataset); 
#dataset='hyperedges-cat-edge-algebra-questions_simplices'; name_file='{0}'.format(dataset);
#dataset='hyperedges-cat-edge-geometry-questions_simplices'; name_file='{0}'.format(dataset);
#dataset='hyperedges-cat-edge-music-blues-reviews'; name_file='{0}'.format(dataset);
#dataset='Mid1'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='Elem1'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='InVS15'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='SFHH'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='LH10'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='LyonSchool'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='Thiers13'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='M_PL_015_ECO_ins';name_file='{0}'.format(dataset);
#dataset='M_PL_062_ECO_ins';name_file='{0}'.format(dataset);
#dataset='M_PL_015_ECO_pl';name_file='{0}'.format(dataset);
#dataset='M_PL_015_ECO_pl';name_file='{0}'.format(dataset);

with open('Data/{0}.json'.format(name_file)) as json_file:
    data = json.load(json_file)
    
data = [data[i] for i in range(len(data)) if len(data[i])>1] #select only interactions of size >=2
for i in range(len(data)): # sort each interaction with growing nodes' IDs (e.g. [2,1] -> [1,2])
    data[i].sort()
data.sort() # sort the interactions
data = list(data for data,_ in itertools.groupby(data)) # remove interactions duplicated
data.sort(key = len) # sort the interactions according to their length
size_max=len(data[-1])

H=xgi.Hypergraph(data)
N=len(H.nodes)

## Simulation

In [ ]:
mu=0.1
n_runs = 1000
l=0.005
theta=0.5
tmax=float(1e3)

dt_inf={}
for i in H.nodes:
    dt_inf[i]=np.zeros(n_runs)

for run_id in tqdm(range(n_runs)):         
    y=discrete_SIS(H,l,mu,dt_inf,run_id,theta,initial_number=1,tmin=0,tmax=tmax,dt=1.0)
   
    for i in H.nodes:
        dt_inf[i][run_id]=dt_inf[i][run_id]/tmax

pk.dump(dt_inf, open('dt_inf_dict_{0}_theta{1}_lambda{2}_mu{3}.pck'.format(dataset,theta,l,mu),'wb'))